# Yandex Algorithm 2018 ML Track
## baseline для Пермского ML-семинара 06.04.2018
должен давать больше 81000 на лидерборде, при подкрутке параметров, озвученной на семинаре - более 84000-85500, что на 06.04.2018 приводит в тор 10 лидерборда

In [49]:
import pandas as pd
import numpy as np

In [50]:
train = pd.read_csv('datasets/yandex/train.tsv', sep='\t', quoting=3, header=None)

In [51]:
test = pd.read_csv('datasets/yandex/public.tsv', sep='\t', quoting=3, error_bad_lines=False, header=None)

In [52]:
test.shape

(9968, 6)

In [53]:
train.fillna('', inplace=True)
test.fillna('', inplace=True)

In [54]:
train.sample(3)

,0,1,2,3,4,5,6,7
56770,163738525330637,почему бы вам не посмотреть на это ?,читательский билет .,"ну конечно , в нём . . . он . . .",4,"по крайней мере , он большой .",bad,0.976371
6648,18333367324612,"я знала , что она больна и что она в плохом со...",кэролайн . . .,"я её дочь , стефан .",5,элисон . . .,bad,0.980096
52976,152405486304269,помогите !,"за мной , мои герои !",матерь божья ! помогите !,0,кто - нибудь на помощь !,good,0.881880


In [55]:
test.sample(3)

,0,1,2,3,4,5
1397,38829940397708,знаю !,"ну , если ты должна ему следовать , советую из...",ты права .,5,умница .
1234,35403964450290,""" это не из - за тебя , а из - за меня "" , так ?",действительно ?,это правда .,4,я знаю .
4703,134945298589002,,что ?,джоан !,4,клэр !


In [56]:
# расставляем значения по формуле хорошесть * confidence
# Good (2)
# Neutral (1)
# Bad (0)

In [57]:
def rank2num(st):
    if st == 'good':
        return 2
    else:
        if st == 'neutral':
            return 1
        else:
            return 0

In [58]:
train['rank'] = train[6].apply(rank2num)
train['target'] = train['rank'] * train[7]
train.head()

,0,1,2,3,4,5,6,7,rank,target
0,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",0,не могу .,good,0.875352,2,1.750703
1,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",1,"нет , звонить буду я .",neutral,0.900968,1,0.900968
2,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",2,"слушай , я не мог уйти .",bad,0.884320,0,0.000000
3,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",3,я не прекращу звонить .,good,0.982530,2,1.965061
4,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",4,я звоню им .,good,0.838054,2,1.676107


In [59]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [60]:
content = train[1].append(train[2]).append(train[3]).append(train[5]).append(test[1]).append(test[2]).append(test[3]).append(test[5])

In [61]:
%%time
tfidf = TfidfVectorizer().fit(content)

CPU times: user 3.62 s, sys: 23.9 ms, total: 3.64 s
Wall time: 3.66 s


In [62]:
t1 = tfidf.transform(train[1])
t2 = tfidf.transform(train[2])
t3 = tfidf.transform(train[3])
t5 = tfidf.transform(train[5])

te1 = tfidf.transform(test[1])
te2 = tfidf.transform(test[2])
te3 = tfidf.transform(test[3])
te5 = tfidf.transform(test[5])

In [63]:
%%time
import scipy
from scipy.sparse import coo_matrix

X_train = scipy.sparse.hstack([t1, t2, t3, t5]).tocsr()
X_test  = scipy.sparse.hstack([te1, te2, te3, te5]).tocsr()

CPU times: user 351 ms, sys: 20 ms, total: 371 ms
Wall time: 384 ms


In [64]:
X_train.shape, X_test.shape

((97533, 172024), (9968, 172024))

In [65]:
y_train = train['target']

In [66]:
train_part_size = int(0.75 * train['target'].shape[0])
X_train_part = X_train[:train_part_size, :]
y_train_part = y_train[:train_part_size]
X_valid =  X_train[train_part_size:, :]
y_valid = y_train[train_part_size:]

In [67]:
X_train_part.shape

(73149, 172024)

In [68]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso

reg = Ridge()

reg.fit(X_train_part, y_train_part)

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [69]:
reg_pred = reg.predict(X_valid)

In [70]:
# я знаю, что метрика ниже - неправильная, но на практике её снижение даёт уверенный рост на лидерборде

In [71]:
%%time
from sklearn.metrics import mean_absolute_error

valid_mae = mean_absolute_error(y_valid, reg_pred)
print(valid_mae)

0.731016807978
CPU times: user 2.49 ms, sys: 0 ns, total: 2.49 ms
Wall time: 2.47 ms


In [72]:
%%time
reg.fit(X_train, y_train)

CPU times: user 6.27 s, sys: 0 ns, total: 6.27 s
Wall time: 6.3 s


Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [73]:
y_test = reg.predict(X_test)

In [74]:
sub = pd.DataFrame()

In [75]:
test.sample(3)

,0,1,2,3,4,5
8502,238596798812654,,,почему ты осталась с сиси после того как ей ст...,5,потому что ее счастье зависит от этого .
8082,225307166540011,я просыпалась от запаха свежих цветов и укола ...,я не боялась .,страх и боль были так . . .,4,это не больно .
2136,58529287634515,"я отремонтирую дыру тоже , но , знаешь , может...",сарай для лодки на мысе трески .,тебе нравится треска ?,5,"да , очень нравится ."


In [76]:
sub['context_id'] = test[0]
sub['reply_id'] = test[4]
sub['rank'] = - y_test

In [77]:
sub.head()

,context_id,reply_id,rank
0,138920940977,0,-0.919949
1,138920940977,1,-1.213151
2,138920940977,2,-0.998704
3,138920940977,3,-1.269976
4,138920940977,4,-1.332131


In [78]:
submission = sub.sort_values(by=['context_id', 'rank'])

In [79]:
del submission['rank']

In [80]:
submission.head()

,context_id,reply_id
4,138920940977,4
3,138920940977,3
1,138920940977,1
5,138920940977,5
2,138920940977,2


In [81]:
test.shape, sub.shape

((9968, 6), (9968, 3))

In [82]:
submission.to_csv('yandex-ml-ridge-baseline-very-simple.tsv',header=None, index=False, sep=' ')